In [ ]:
import pandas as pd
import json
import requests
import calendar
import datetime as dt
from dateutil.relativedelta import *

In [ ]:
def do_request_transaksi(f,d,so):
  url = 'http://sim.rs/simrs/inventori/laporan_inventori/pub_txt_logfar/'
  myobj = {'tgl_dari': f,'tgl_sampai': d,'kode_so': so}

  x = requests.post(url, data = myobj)

  return x.json()

def get_stok_akhir(y,m):
  url = 'http://sim.rs/devapi/rsp/invsofarmasi/show_stok_so/'
  myobj = {'y': y,'m':m}

  x = requests.post(url, data = json.dumps(myobj),headers={'Content-Type': 'application/json', 'Accept':'application/json'})
  
  return x.json()

def get_ket_bast(l):
  url = 'http://sim.rs/devapi/rsp/invpenerimaanfarmasi/penerimaan_bast_bku/'
  myobj = {'po': l}

  x = requests.post(url, data = json.dumps(myobj),headers={'Content-Type': 'application/json', 'Accept':'application/json'})
  
  return x.json()

def get_list_simak_date(y,lim_range = 13):
    r = []
    for i in range(1,lim_range,1):
        e = calendar.monthrange(y, i)[1]
        m = str(i).zfill(2)
        r.append(['01-'+m+'-'+str(y)+' 00:00',str(e)+'-'+m+'-'+str(y)+' 23:59',str(y)[2]+str(y)[3]+m])
    return r

def komparasiDenganDesc(D1,D2,r=0):
    return pd.merge(D1,D2,on=r,how='outer',indicator=True)

  

In [118]:
R = []
mmax = 3

for i in range(1,(mmax+1)):
    d = (dt.datetime.today()+ relativedelta(months=(-1*i))).strftime('%y%m')
    R.append(pd.DataFrame(get_stok_akhir(d[0]+d[1],d[2]+d[3])))



In [119]:

frst = R[0][['c','e','n']][(R[0]['e'] > 0)&(R[0]['k'] == 0)]
newDF = pd.DataFrame()
for j in range(1,mmax):
    nowDF = R[j][['c','k']][(R[j]['k'] == 0)]
    if j > 1 : newDF = pd.merge(newDF,nowDF,how='inner')
    else : newDF = nowDF
slowDF = pd.merge(frst,newDF,how='inner')

In [120]:
slowDF['kode'] = slowDF.iloc[:].astype(str).apply(lambda x: '|'+x['c']+'|' ,axis=1)
slowDF.to_csv("slowDF.csv",sep='\t')